<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/%E7%9F%A5%E4%B9%8E%E7%AC%94%E8%AE%B0%EF%BC%9Aattention%E6%9C%BA%E5%88%B6%E8%AF%A6%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1.RNN结构的局限
&emsp;&emsp;以机器翻译为例，通常使用encoder-decoder结构，即encoder首先将输入的句子转换为一个定长的向量，然后decoder再将这个向量翻译成目标语言的文字。encoder和decoder一般都采用LSTM或者GRU等。
![替代文字](https://pic1.zhimg.com/80/v2-60645bb5a5777b7bcee90c78de34eb00_720w.jpg)

&emsp;&emsp;如上图所示，encoder端RNN将输入语句换换到最后一个hidden vector中，并将其作为decoder端的初始hidden vector，然后利用decoder解码成相应语言的文字。
&emsp;&emsp;上图结构存在的问题：
* 当输入序列太长，RNN结构存在梯度消失的问题
* 将输入的序列转化为定长向量以难以有效的保存信息，随着输入序列的增长，效果会更差。

## 2. attention机制

![替代文字](https://pic4.zhimg.com/80/v2-a5f8a19c6d89422fe7d8a74087088b37_720w.jpg)

* 首先得到encoder端的hidden state ($h_1,h_2,\ldots,h_T$)
* 计算注意力得分
假设当前decoder的hidden state是$s_{t-1}$，计算每一个输入位置$j$与当前输出位置的关联性$e_{tj}=a(s_{t-1},h_j)$,相应的向量形式为：$\overrightarrow{e_t}=(a(s_{t-1},h_1),\ldots,a(s_{t-1,h_T}))$。其中，a是一种相关性运算符，例如常见的有点乘形式$\overrightarrow{e_t}=\overrightarrow{s_{t-1}}^T\overrightarrow{h}$、加权点乘
$\overrightarrow{e_t}=\overrightarrow{s_{t-1}}^T W \overrightarrow{h}$、加和$\overrightarrow{e_t}=\overrightarrow{v}^Ttanh(W_1\overrightarrow{h}+W_2\overrightarrow{s_{t-1}})$等等。\
对$\overrightarrow{s_t}$进行softmax操作，得到attention的分布，$\overrightarrow \alpha _t=softmax(\overrightarrow{s_t})$,展开形式为:$\alpha _{tj}=\frac{exp(e_{tj})}{\sum _{k=1}^Texp(e_{tk})}$

* 利用$\overrightarrow{\alpha _t}$计算context vector:
$\overrightarrow{c _t}=\sum_{j=1}^T \alpha _{tj}h_j$

* 根据$s_{t-1},y_{t-1},c_t$计算decoder的下一个hidden state以及输出：
$$s_t=f(s_{t-1},y_{t-1},c_t)$$
$$p(y_t|y_1,\dots,y_{t-1},x)=g(y_{i-1},s_i,c_i)$$

## 3. attention机制总结

* 关键操作是计算encoder与decoder state之间的关联性的权重，得到attention分布，从而对于当前输出位置得到比较重要的输入位置的权重，在预测输出时相应的会占较大的比重。
* attention机制打破了只能利用encoder最终单一固定结果向量的限制，从而模型可以集中在所有对于下一个目标单词重要的输入信息上，使模型效果可以得到极大的改善。另外一个优点是可以可视化观察attention权重矩阵的变化，可以更好地知道哪部分翻译对应哪部分源文字，有助于更好地理解模型工作机制。